# 05. 통계검정 (T-test)

## 목적
- 고위험 vs 정상 그룹 간 지표군별 차이 검정
- Cohen's d로 효과 크기 측정

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

result_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석"
cluster_result = pd.read_csv(f"{result_dir}/클러스터링_결과_완전판.csv")

data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
df_full = pd.read_csv(data_path)

df = cluster_result.merge(df_full, on='가맹점구분번호', how='left')
df['위험그룹'] = df['cluster'].map({0: '고위험', 1: '정상', 2: '고위험'})

print(f"그룹별 분포:")
print(df['위험그룹'].value_counts())

그룹별 분포:
위험그룹
고위험    88578
정상       825
Name: count, dtype: int64


## T-검정 수행

In [2]:
# 운영지표 샘플
operating_cols = ['당월_매출_금액', '당월_매출_건수', '배달매출금액 비율']
existing_cols = [c for c in operating_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]

results = []
high_risk = df[df['위험그룹'] == '고위험']
normal = df[df['위험그룹'] == '정상']

for col in existing_cols:
    hr_vals = high_risk[col].dropna()
    nm_vals = normal[col].dropna()
    
    if len(hr_vals) >= 30 and len(nm_vals) >= 30:
        t_stat, p_value = ttest_ind(hr_vals, nm_vals, equal_var=False)
        mean_diff = hr_vals.mean() - nm_vals.mean()
        pooled_std = np.sqrt((hr_vals.std()**2 + nm_vals.std()**2) / 2)
        cohens_d = mean_diff / pooled_std if pooled_std > 0 else 0
        
        results.append({
            '변수': col,
            '고위험_평균': hr_vals.mean(),
            '정상_평균': nm_vals.mean(),
            'Cohen_d': cohens_d,
            'p값': p_value,
            '유의성': 'Yes' if p_value < 0.05 else 'No'
        })

results_df = pd.DataFrame(results)
print("\n[T-검정 결과]")
print(results_df)


[T-검정 결과]
          변수        고위험_평균         정상_평균   Cohen_d            p값  유의성
0   당월_매출_금액  7.971027e+08  1.065620e+09 -0.130068  2.425822e-03  Yes
1   당월_매출_건수  2.952727e+04  3.889582e+04 -0.119335  2.931431e-03  Yes
2  배달매출금액 비율  9.404083e+00  2.007321e+01 -0.365095  1.188648e-17  Yes


### 통계적 해석
- p < 0.05: 통계적으로 유의한 차이
- Cohen's d > 0.8: 큰 효과 크기
- 고위험 그룹의 특성 파악